In [1]:
import torch
import torchvision.transforms as transforms
import timm
from tqdm import tqdm
from torch.utils.data import Dataset, Subset, DataLoader, Sampler
import glob
import os
from PIL import Image
from collections import defaultdict, Counter
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
import json
import math

os.environ["LOKY_MAX_CPU_COUNT"] = "6"

SEED = 42

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

cifar_dataset = load_dataset("uoft-cs/cifar10")
cifar_dataset

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})

In [3]:
from datasets import load_dataset

mnist_dataset = load_dataset("mnist")
mnist_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(10),
    # transforms.ColorJitter(0.1, 0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], 
                         std=[0.2470, 0.2435, 0.2616])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], 
                         std=[0.2470, 0.2435, 0.2616])
])

In [11]:
class custom_dataset(Dataset):
    def __init__(self, vanila_dataset, image_indexes, transform):
        self.dataset = vanila_dataset
        self.image_indexes = image_indexes

        self.transform = transform

    def __len__(self):
        return len(self.image_indexes)
    
    def __getitem__(self, index):
        selected_index = self.image_indexes[index]
        
        _image = self.dataset[selected_index]['img']
        _image = self.transform(_image)

        _label = self.dataset[selected_index]['label']

        return _image, _label, selected_index

In [7]:
def worker_init_fn(worker_id):
    seed = worker_id + SEED

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    
    return

In [8]:
class custom_data_factory():
    def __init__(self, train_dataset, test_dataset, sample_num, train_transform, valid_transform):
        self.train_data_dict = {}

        for i in range(0, train_dataset.num_rows):
            _index = i
            _label = train_dataset[i]['label']
            if self.train_data_dict.get(_label) is None:
                self.train_data_dict[_label] = []

            self.train_data_dict[_label].append(_index)

        self.train_datset = train_dataset
        self.test_dataset = test_dataset

        self.selected_items = {_label: random.choices(item_index, k=min(len(item_index), sample_num)) for _label, item_index in self.train_data_dict.items()}

        self.sample_num = sample_num
        self.train_transform = train_transform
        self.valid_transform = valid_transform

        self.losses = []

    def get_data_set(self):
        _item_list =  sum(self.selected_items.values(), [])
        random.shuffle(_item_list)
        
        self.train_set = custom_dataset(self.train_datset, _item_list, self.train_transform)
        self.valid_set = custom_dataset(self.test_dataset, [i for i in range(0, 10000)], self.valid_transform)

        print(f'Train set: {len(self.train_set.image_indexes)}')
        print(f'Valid set: {len(self.valid_set.image_indexes)}')

        return self.train_set, self.valid_set
    
    def get_data_loader(self):
        train_loader = DataLoader(
            self.train_set, pin_memory=True,
            batch_size=32,
            num_workers=0,
            drop_last=True
        )

        valid_loader = DataLoader(
            self.valid_set, pin_memory=True,
            batch_size=16,
            num_workers=0,
            drop_last=True
        )

        return train_loader, valid_loader
    
    def collect_losses(self, _paths, _labels, _losses):
        for _path, _label, _loss in zip(_paths, _labels, _losses):
            self.losses.append((_path.item(), _label.item(), _loss.item()))

    def renew_data_loader(self, resample_ratio):
        selected_items_dict = {}
        for _path, _label, _loss in self.losses:
            if selected_items_dict.get(_label) == None:
                selected_items_dict[_label] = []

            selected_items_dict[_label].append((_path, _loss))
        
        selected_items = {}
        for _label, _path_list in selected_items_dict.items():
            _paths, _weights = zip(*_path_list)
            _weights = [(_weight - min(_weights))/(max(_weights)-min(_weights)) for _weight in _weights]

            selected_items[_label] = random.choices(_paths,
                                                    k=int(self.sample_num * (1-resample_ratio)),
                                                    weights=_weights)
        
        unselected_items = {_label: random.sample([_path for _path in _path_list if _path not in selected_items_dict[_label]],
                                                  k=int(self.sample_num * resample_ratio)) for _label, _path_list in self.train_data_dict.items()}

        self.selected_items = {}
        for (_label, _selected_path_list), (_label, _unselected_path_list) in zip(selected_items.items(), unselected_items.items()):
            self.selected_items[_label] = []
            self.selected_items[_label].extend(_selected_path_list)
            self.selected_items[_label].extend(_unselected_path_list)

        # print(self.selected_items)

        self.losses = []

        self.get_data_set()

        return self.get_data_loader()

In [12]:
def evaluate(model, valid_loader):
    model.eval()
    all_probs, all_labels = [], []
    
    with torch.no_grad():
        for images, labels, selected_indexes in tqdm(valid_loader, desc=f'Validating', leave=False):
            images, labels = images.to('cuda'), labels.to('cuda')
            
            outputs = model(images)

            probs = F.softmax(outputs, dim=1)

            all_probs.append(probs.cpu().numpy().flatten())
            all_labels.append(labels.cpu().numpy())
    
    all_probs_roc = np.array(all_probs).reshape(-1, 10)
    all_probs = np.array([np.argmax(_pred) for _pred in all_probs_roc])
    all_labels = np.array(all_labels).flatten()

    return {
        'accuracy': accuracy_score(all_labels, all_probs),
        'precision': precision_score(all_labels, all_probs, average='weighted', zero_division=0),
        'recall': recall_score(all_labels, all_probs, average='weighted', zero_division=0),
        'f1': f1_score(all_labels, all_probs, average='weighted', zero_division=0),
        'roc_auc':roc_auc_score(all_labels, all_probs_roc, multi_class='ovr'),
        # 'confusion_matrix': confusion_matrix(all_labels, all_probs)
    }

In [15]:
epochs = 400
object_dataset = 'cifar'

In [ ]:
import timm


base_model = timm.create_model('resnet50', pretrained=False, num_classes=10).to('cpu')

for sample_nums in [1000, 1250, 2500]:
    datafactory = custom_data_factory(
        eval(f"{object_dataset}_dataset['train']"),
        eval(f"{object_dataset}_dataset['test']"),
        sample_num=sample_nums,
        train_transform=train_transform,
        valid_transform=valid_transform
    )

    model = timm.create_model('resnet50', pretrained=False, num_classes=10)
    model.load_state_dict(base_model.state_dict())

    model.to('cuda')
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, weight_decay=1e-6)

    train_history = {}
    criterion = nn.CrossEntropyLoss()

    datafactory.get_data_set()
    train_loader, valid_loader = datafactory.get_data_loader()

    for epoch in range(epochs):
        model.train()
        total_losses = []

        for images, labels, selected_index in tqdm(train_loader, desc=f'Training epoch: {epoch+1}/200', leave=False):
            images = images.to('cuda')
            labels = labels.to('cuda')

            optimizer.zero_grad()
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            
            loss.mean().backward()
            optimizer.step()

            total_losses.append(loss.mean().item())
            
            datafactory.collect_losses(selected_index, labels.cpu(), [loss.cpu()])

        valid_results = evaluate(model, valid_loader=valid_loader)
        valid_results['loss'] = np.mean(total_losses)
        train_history[epoch] = valid_results

        train_loader, valid_loader = datafactory.renew_data_loader(resample_ratio=0.7)

        json_path = os.path.join(os.getcwd(), rf'results/{object_dataset}/collect_losses_train_history_{sample_nums}.json')
        os.makedirs(os.path.dirname(json_path), exist_ok=True)

        with open(json_path, 'w', encoding='utf-8-sig') as json_file:
             json.dump(train_history, json_file, indent=2, ensure_ascii=False) 

Train set: 10000
Valid set: 10000


Train set: 10000
Valid set: 10000


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\workspace\\paper_1\\results/cifar/collect_losses_train_history_1000.json'

In [16]:
# model = timm.create_model('resnet50', pretrained=False, num_classes=10)
model = torch.load('base_model.pt', weights_only=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

model.to('cuda')

train_history = {}
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, weight_decay=1e-6)

train_set = custom_dataset(mnist_dataset['train'], [i for i in range(0, 50000)], train_transform)
valid_set = custom_dataset(mnist_dataset['test'], [i for i in range(0, 10000)], valid_transform)

train_loader = DataLoader(
    train_set, pin_memory=True,
    batch_size=64,
    num_workers=0,
    drop_last=True
)

valid_loader = DataLoader(
    valid_set, pin_memory=True,
    batch_size=32,
    num_workers=0,
    drop_last=True
)

criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_losses = []

    for images, labels, selected_index in tqdm(train_loader, desc=f'Training epoch: {epoch+1}/200', leave=False):
        images = images.to('cuda')
        labels = labels.to('cuda')

        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.mean().backward()
        optimizer.step()

        total_losses.append(loss.mean().item())

    valid_results = evaluate(model, valid_loader=valid_loader)
    valid_results['loss'] = np.mean(total_losses)
    train_history[epoch] = valid_results
    
    with open(os.path.join(os.getcwd(), f'results/mnist_base_train_history.json'), 'w', encoding='utf-8-sig') as json_file:
        json.dump(train_history, json_file, indent=2, ensure_ascii=False)

KeyboardInterrupt: 